In [1]:
### IMPORT
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
import gensim

### Load Dataset (Preprocessed)
df_train = pd.read_excel("https://raw.githubusercontent.com/SNMHZ/Drug_Recommendation/master/dataset/5/lem_train2.xlsx")
df_test = pd.read_excel("https://raw.githubusercontent.com/SNMHZ/Drug_Recommendation/master/dataset/5/lem_test2.xlsx")

### 결측값 제거
df_train = df_train.dropna(how='any')
df_test = df_test.dropna(how='any')

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [49]:
### 각종 함수
def findAVGVec(sentence):
  '''
  평균 벡터 구해주는 메서드
  findAVFVec((str)sentence)
  '''
  result = np.array( [0]*300, dtype=np.float64 )
  for wordVector in sentence:
    result += wordVector
  
  result = result/len(sentence)
  return result

def kill_notword(_list):
  '''
  문장단위로 나눌 때 문장의 길이가 1 이하인것 삭제
  -> 공백과 아무것도 없는 문장 삭제
  kill_notword((list)_list)
  '''
  for i in reversed(range(len(_list))):
    if len(_list[i])<=1:
      del _list[i]

def makeInputVector(data):
  '''
  한 문장 씩 분리
  문장 A, B, ... E 로 이루어진 리뷰 데이터가 condition이 ADHD라면
  [[A, B, .., E], ..., [1490395번째 리뷰를 문장으로 나눈 리스트]]
  [ADHD, ..., 1490395번째]
  '''
  for review, condition in enuramte(data):
    result = np.array()
    for sentence in review:
      result.append()

def softmax(x):
  '''
  softmax(numpy_array)
  '''
  fx = np.exp(x)/np.sum(np.exp(x))
  return fx

In [50]:
### 워드 임베딩 모델 불러오는 부분
### 현재 사용하는 임베딩 모델은 (300,1)
### 

model = Word2Vec.load("https://raw.githubusercontent.com/SNMHZ/Drug_Recommendation/master/Web/flask/model3_word2vec.model")

In [51]:
### df['splited_list']에 한 review를 문장별로 나누어 list에 추가
### !,?,. 을 기준으로 문장을 나누고 리스트를 만들어 df에 저장

splited_list = []
for review in df_train['review']:
  try:
    splited = re.split('\!|\?|\.', review)
    if splited != np.nan:
      splited_list.append( splited )
  except:
    splited_list.append( [review] )

for splited in splited_list:
  try:
    kill_notword(splited)
  except:
    pass

df_train['sentences_list']=splited_list

In [52]:
### sentences_list의 문장별로 vector화
### 문장 벡터의 size는 300,1
vectored_list = []
for sentence_list in df_train['sentences_list']:
  sentence_vector_list = []
  for sentence in sentence_list:
    word_vector_list = []
    sentence_splited = sentence.split(' ')
    for word in sentence_splited:
      try:
        word_vector_list.append(model.wv.get_vector(word))
      except:
        pass
    avgvec = findAVGVec(word_vector_list)
    if avgvec[0] != np.nan:
      sentence_vector_list.append(findAVGVec(word_vector_list))
  vectored_list.append(sentence_vector_list)

df_train['vectored_list'] = vectored_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [ ]:
### 컨디션 Labeling

import numpy as np

target_vector_row_size = len(df_train['condition'].unique())
data = df_train['condition'].unique()

target_list = []

#### 각 review행에 대응되는 label vector 만들기
for data_index in range(0, df_train.shape[0]):
    target_condition = df_train['condition'].iloc[data_index]
    target_vector = np.zeros((1, target_vector_row_size), dtype='object')
    target_vector[0][np.where(data == target_condition)] = 1
    target_list.append(target_vector)

      # 특정 인덱스의 데이터가 없는 경우가 있음(예 : 인덱스 순서가 103 -> 105 바로 진행되는 경우)
print(len(target_list))
#df_train['target_vector'] = np.nan

#### label vector를 데이터 프레임에 붙이기.
for data_index in range(0, df_train.shape[0]):
    df_train['target_vector'].iloc[data_index] = target_list[data_index]

In [ ]:
### 문장으로 컨디션 학습
### RNN RF MLP LSTM TEXT-CNN

In [ ]:
### 임시 입력 df_test['review'][354]
msg='work incredibly well prevention g per day oral herpes occasional mean year last current g disappears inside week . manage transfer cold sore genital area work well still aid outbreak often take week disperse . get bulk powder instead tablet . cost greatly reduce need mix splash juice drink '

In [71]:
### 챗봇의 입력을 벡터로 변환

msg_list=re.split('\!|\?|\.', msg)
kill_notword(msg_list)

msg_vectored = []
for sentence in msg_list:
  word_vector_list = []
  sentence_splited = sentence.split(' ')
  for word in sentence_splited:
    try:
      word_vector_list.append(model.wv.get_vector(word))
    except:
      pass
  avgvec = findAVGVec(word_vector_list)
  if avgvec[0] != np.nan:
    msg_vectored.append(findAVGVec(word_vector_list))


In [72]:
print(len(msg_list))
print(msg_list)
print(len(msg_vectored))
print(msg_vectored)

4
['work incredibly well prevention g per day oral herpes occasional mean year last current g disappears inside week ', ' manage transfer cold sore genital area work well still aid outbreak often take week disperse ', ' get bulk powder instead tablet ', ' cost greatly reduce need mix splash juice drink ']
4
[array([ 1.03179018e-01, -2.45445504e-01,  3.23494269e-01, -6.42024326e-02,
        3.45864561e-03,  6.94503416e-02, -2.44618328e-01,  1.81004266e-01,
       -1.50515839e-01, -7.73345413e-02, -2.84442448e-01,  2.84995275e-01,
        2.61136718e-01, -2.03176438e-01,  2.54846967e-02, -5.46649093e-02,
       -1.26919477e-01,  2.77585078e-01, -1.05128646e-01, -4.48693122e-02,
        5.76203072e-02, -2.02256008e-01,  3.37171072e-01,  1.97319275e-01,
        3.86570611e-03, -2.87136755e-01, -7.92080406e-02, -4.35947537e-01,
       -3.46632677e-01,  1.22848888e-01, -1.22239015e-01,  2.87959736e-02,
        8.79693340e-02,  1.35712834e-01,  1.17467128e-01, -2.64565165e-01,
        1.08103